In [3]:
import os
import re
import time
import numpy as np
import pandas as pd
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import openai
from dotenv import load_dotenv
import pickle
import hashlib

In [4]:
# Load .env file
load_dotenv()

# Set API key from environment
openai.api_key = os.getenv("OPENAI_API_KEY")

if openai.api_key is None:
    raise ValueError("OpenAI API key not found. Make sure you have a .env file with OPENAI_API_KEY set.")

In [5]:
def encode_texts_openai(texts, cache_path="embeddings_cache.pkl", model="text-embedding-3-large"):
    print("\nEncoding texts using OpenAI model...")
    
    # Load cache if exists
    if os.path.exists(cache_path):
        with open(cache_path, "rb") as f:
            cache = pickle.load(f)
    else:
        cache = {}

    embeddings = []
    new_count = 0

    for text in tqdm(texts, desc="Embedding"):
        key = hashlib.md5(text.encode('utf-8')).hexdigest()
        if key in cache:
            embeddings.append(cache[key])
        else:
            try:
                response = openai.embeddings.create(input=text, model=model)
                vec = response.data[0].embedding
                cache[key] = vec
                embeddings.append(vec)
                new_count += 1
            except Exception as e:
                print(f"OpenAI API failed on: {text[:50]}...\n{e}")
                vec = [0.0] * 1536  # fallback, dim of text-embedding-3-small
                embeddings.append(vec)

    # Save updated cache
    with open(cache_path, "wb") as f:
        pickle.dump(cache, f)

    print(f"Embedding complete. {new_count} new vectors generated, {len(texts) - new_count} loaded from cache.")
    return np.array(embeddings)

In [6]:
def load_data(tb1_path, tb2_path):
    table1 = pd.read_csv(tb1_path)
    table2 = pd.read_csv(tb2_path)
    return table1, table2

def preprocess(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'[^\w\s-]', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def preprocess_table(df, columns_to_concat):
    # Fill NaN for each selected column
    for col in columns_to_concat:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in dataframe.")
        df[col] = df[col].fillna('')

    # Apply preprocess and concatenate
    processed_text = ''
    for col in columns_to_concat:
        processed_text += df[col].apply(preprocess) + ' '
    
    df['processed'] = processed_text.str.strip()
    df['processed'] = df['processed'].replace('', 'unknown')
    return df

def preprocess_tables(table1, table2, table1_columns, table2_columns):
    table1 = preprocess_table(table1, table1_columns)
    table2 = preprocess_table(table2, table2_columns)
    return table1, table2

def encode_texts(model, texts, batch_size=64):
    print("\nEncoding texts into embeddings...")
    start_time = time.time()
    embeddings = model.encode(texts, show_progress_bar=True, batch_size=batch_size)
    elapsed = time.time() - start_time
    print(f"Embedding generation completed in {elapsed:.2f} seconds.")

    if np.isnan(embeddings).any():
        print("Warning: NaN values detected in embeddings - replacing with zeros")
        embeddings = np.nan_to_num(embeddings)
    return embeddings

def normalize_embeddings(embeddings):
    norms = np.linalg.norm(embeddings, axis=1)
    norms[norms == 0] = 1e-10
    return embeddings / norms[:, np.newaxis]

def build_faiss_index(embeddings, batch_size=1000):
    print("\nBuilding FAISS index...")
    start_time = time.time()
    dimension = embeddings.shape[1]
    if dimension == 0:
        raise ValueError("Embedding dimension is 0 - check your input data")
    
    try:
        index = faiss.IndexFlatIP(dimension)
        for i in tqdm(range(0, len(embeddings), batch_size), desc="Adding batches to FAISS"):
            batch = embeddings[i:i+batch_size]
            if not np.isnan(batch).any():
                index.add(batch)
            else:
                print(f"Skipping batch {i} due to NaN values")
        elapsed = time.time() - start_time
        print(f"FAISS index built in {elapsed:.2f} seconds.")
        return index
    except Exception as e:
        print(f"Error creating FAISS index: {e}")
        print("Using brute-force matching instead")
        return None

def find_matches(query_embeddings, table1, target_df, target_embeddings, index=None, top_k=30):
    if index is None:
        raise ValueError("FAISS index is not available. Matching cannot proceed.")
    print("\nFinding matches...")
    matches = []
    start_time = time.time()
    for i, query_embedding in enumerate(tqdm(query_embeddings, desc="Matching queries")):
        if np.isnan(query_embedding).any():
            print(f"Skipping query {i} due to NaN values")
            continue

        if index:
            query_embedding = query_embedding.reshape(1, -1)
            distances, indices = index.search(query_embedding, top_k)
            distances = distances[0]
            indices = indices[0]
        # else:
        #     similarities = np.dot(target_embeddings, query_embedding)
        #     indices = np.argsort(similarities)[-top_k:][::-1]
        #     distances = similarities[indices]

        for score, idx in zip(distances, indices):
            matches.append({
                'left_id': table1.iloc[i]['ID'],
                'right_id': target_df.iloc[idx]['ID'],
                'similarity_score': score,
                'table1_text': table1.iloc[i]['processed'],
                'table2_text': target_df.iloc[idx]['processed']
            })
    elapsed = time.time() - start_time
    print(f"Matching completed in {elapsed:.2f} seconds.")
    return matches

def save_results(matches_df, index=None, matches_file="entity_matches.csv", index_file="entity_matching_index.faiss"):
    matches_df.to_csv(matches_file, index=False)
    print(f"\nSaved matches to {matches_file}.")
    if index:
        faiss.write_index(index, index_file)
        print(f"Saved FAISS index to {index_file}.")



In [7]:
def main():
    # File paths
    tb1_path = os.path.join("files", "amazon.csv")
    tb2_path = os.path.join("files", "best_buy.csv")

    # Load and preprocess data
    table1, table2 = load_data(tb1_path, tb2_path)
    
    table1_columns = ["Brand", "Name"]        
    table2_columns = ["Brand", "Name"]     

    # table1_columns = ["Brand", "Name", "Features"]        
    # table2_columns = ["Brand", "Name", "Description"]     
    
    table1, table2 = preprocess_tables(table1, table2, table1_columns, table2_columns)

    # Load model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    # model = SentenceTransformer("all-mpnet-base-v2") model is too big

    # Generate and normalize embeddings
    # embeddings1 = encode_texts(model, table1['processed'].tolist())
    # embeddings2 = encode_texts(model, table2['processed'].tolist())
    # embeddings1 = normalize_embeddings(embeddings1)
    # embeddings2 = normalize_embeddings(embeddings2)


    # use the openai
    # embeddings1 = encode_texts_openai(table1['processed'].tolist(), cache_path="table1_embeddings.pkl")
    # embeddings2 = encode_texts_openai(table2['processed'].tolist(), cache_path="table2_embeddings.pkl")
    
    # embeddings1 = normalize_embeddings(embeddings1)
    # embeddings2 = normalize_embeddings(embeddings2)

    # Build FAISS index
    index = build_faiss_index(embeddings2)

    # Find matches
    matches = find_matches(embeddings1, table1, table2, embeddings2, index=index, top_k=30)
    matches_df = pd.DataFrame(matches)

    # Display some results
    print("\nTop matches between tables:")
    print(matches_df.sort_values(by='similarity_score', ascending=False).head())

    # Save results
    save_results(matches_df, index)

if __name__ == "__main__":
    main()


Encoding texts using OpenAI model...


Embedding: 100%|███████████████████████████████████████████████| 4259/4259 [27:26<00:00,  2.59it/s]


Embedding complete. 4217 new vectors generated, 42 loaded from cache.

Encoding texts using OpenAI model...


Embedding: 100%|███████████████████████████████████████████████| 5001/5001 [33:00<00:00,  2.53it/s]


Embedding complete. 4639 new vectors generated, 362 loaded from cache.

Building FAISS index...


Adding batches to FAISS: 100%|██████████████████████████████████████| 6/6 [00:00<00:00, 143.76it/s]


FAISS index built in 0.05 seconds.

Finding matches...


Matching queries: 100%|███████████████████████████████████████| 4259/4259 [00:13<00:00, 324.54it/s]


Matching completed in 13.12 seconds.

Top matches between tables:
       left_id  right_id  similarity_score  \
11490      384       449          0.984726   
59610     1988      3563          0.981807   
30180     1007       294          0.963221   
28530      952      3225          0.961034   
67950     2266      2419          0.956337   

                                             table1_text  \
11490  lenovo lenovo - g50 156 laptop - intel core i3...   
59610  other 3m privacy filter for widescreen laptop ...   
30180  dell dell - inspiron 156 4k ultra hd touch-scr...   
28530  other cooler master notepal x3 - gaming laptop...   
67950  other solo pro 173 laptop backpack black pro742-4   

                                             table2_text  
11490  lenovo lenovo g50 156 laptop intel core i3 4gb...  
59610  other 3m privacy filter for widescreen laptop ...  
30180  dell dell inspiron 156 4k ultra hd touchscreen...  
28530  other cooler master notepal x3 gaming laptop c...  
6